# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import copy

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read in cities.csv file to data frame
weather_data = pd.read_csv("output_data/cities.csv")
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,53.01,98,75,8.03,AU,2021-04-19 16:18:58
1,Lebu,-37.6167,-73.6500,58.35,95,100,15.99,CL,2021-04-19 16:18:52
2,Chui,-33.6971,-53.4616,64.98,79,0,10.04,UY,2021-04-19 16:23:11
3,Saint Anthony,45.0205,-93.2180,39.00,44,90,11.50,US,2021-04-19 16:23:11
4,Kalmunai,7.4167,81.8167,78.31,83,93,1.83,LK,2021-04-19 16:23:12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[['Lat', 'Lng']]

# Store Humidity
humidity = weather_data['Humidity'].astype(float)

In [7]:
# Customize the size of the figure
figure_layout = {'width': '600px',
                'height': '500px',
                'border': '1px solid black',
                'padding': '1px',
                'margin': '0 auto 0 auto'
                }
fig = gmaps.figure(layout=figure_layout)

# Create a humidity Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300,
                                 point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_weather = weather_data.loc[(weather_data['Max Temp'] < 80) & (weather_data['Max Temp'] > 70)
                               & (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0)]
ideal_weather.dropna()
ideal_weather.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
159,Sakakah,29.9697,40.2064,73.40,17,0,9.22,SA,2021-04-19 16:21:31
186,Roebourne,-20.7833,117.1333,75.92,53,0,7.78,AU,2021-04-19 16:24:19
246,Hurghada,27.2574,33.8129,78.80,29,0,7.65,EG,2021-04-19 16:24:42
285,Amambai,-23.1042,-55.2258,70.21,55,0,7.96,BR,2021-04-19 16:24:57
292,Biloela,-24.4167,150.5000,72.18,58,0,3.60,AU,2021-04-19 16:25:00
310,San Lorenzo,-25.3333,-57.5333,75.20,78,0,4.61,PY,2021-04-19 16:23:11
377,Port Hedland,-20.3167,118.5667,71.60,78,0,5.75,AU,2021-04-19 16:22:03


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = copy.deepcopy(ideal_weather)
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
159,Sakakah,29.9697,40.2064,73.40,17,0,9.22,SA,2021-04-19 16:21:31,
186,Roebourne,-20.7833,117.1333,75.92,53,0,7.78,AU,2021-04-19 16:24:19,
246,Hurghada,27.2574,33.8129,78.80,29,0,7.65,EG,2021-04-19 16:24:42,
285,Amambai,-23.1042,-55.2258,70.21,55,0,7.96,BR,2021-04-19 16:24:57,
292,Biloela,-24.4167,150.5000,72.18,58,0,3.60,AU,2021-04-19 16:25:00,
310,San Lorenzo,-25.3333,-57.5333,75.20,78,0,4.61,PY,2021-04-19 16:23:11,
377,Port Hedland,-20.3167,118.5667,71.60,78,0,5.75,AU,2021-04-19 16:22:03,


In [10]:
# Set up a parameters dictionary
params = {
          'radius': 5000,
          'types': 'lodging',
          'keyword': 'hotel',
          'key': g_key
         }
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    # Get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']

    # Change location each iteration while leaving original params in place
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print(f"{row['City']} not found skipping...") 
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
159,Sakakah,29.9697,40.2064,73.40,17,0,9.22,SA,2021-04-19 16:21:31,Alnuzl Special Hotel
186,Roebourne,-20.7833,117.1333,75.92,53,0,7.78,AU,2021-04-19 16:24:19,Latitude20 Roebourne Village
246,Hurghada,27.2574,33.8129,78.80,29,0,7.65,EG,2021-04-19 16:24:42,Marina Star Hotel
285,Amambai,-23.1042,-55.2258,70.21,55,0,7.96,BR,2021-04-19 16:24:57,Hotel Royal em Amambai
292,Biloela,-24.4167,150.5000,72.18,58,0,3.60,AU,2021-04-19 16:25:00,Biloela Hotel
310,San Lorenzo,-25.3333,-57.5333,75.20,78,0,4.61,PY,2021-04-19 16:23:11,Factoria Hotel
377,Port Hedland,-20.3167,118.5667,71.60,78,0,5.75,AU,2021-04-19 16:22:03,The Esplanade Hotel


In [11]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Customize size of the figure
figure_layout = {'width': '600px',
                'height': '500px',
                'border': '1px solid black',
                'padding': '1px',
                'margin': '0 auto 0 auto'
                }
fig = gmaps.figure(layout=figure_layout)

# Add heat layer
fig.add_layer(heat_layer)

# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…